# 1) Cargar la data

In [1]:
import os, pickle
import pandas as pd
from tabulate import tabulate

In [11]:
def leer_nombre_ficheros(elementos_SQ, temporalidades):
    """
    Encuentra los elementos anteriores al primer match entre la lista de elementos_SQ y temporalidades.

    Args:
        elementos_SQ (list): Lista de elementos a verificar.
        temporalidades (list): Lista de temporalidades a buscar.

    Returns:
        list: Lista de listas que contienen activos encontrados dentro del elemento en elementos_SQ.
        list: Lista de listas que contienen las temporalidades encontradas en cada elemento de elementos_SQ.
    """
    id_busqueda = []
    activos_encontrados = []
    temporalidades_encontradas = []
    
    for elemento in elementos_SQ:
        elementos_previos = []
        primer_elemento = elemento.split('_')[0]
        
        for temporalidad in temporalidades:
            index = elemento.find(temporalidad)
            if index != -1:
                elementos_previos.extend(elemento[:index].split('_')[1:-1])
                temporalidades_encontradas.append(elemento[index:].split('_')[:-1])
                break
        
        id_busqueda.append(primer_elemento)
        activos_encontrados.append(elementos_previos)
    
    return id_busqueda, activos_encontrados, temporalidades_encontradas

# DECORADOR
def styler(func):
    def wrapper(*args, **kwargs):
        df = func(*args, **kwargs)
        # formato para la impresion con estilo
        keys= list(df.columns)
        values = ['{:.2f}' for i in range(len(keys))]
        format_dict = dict(zip(keys, values))
        return df.style.format(format_dict).background_gradient(cmap='PRGn')
    return wrapper

# SE UTILIZA PARA VISUALIZAR LOS DATOS DEL OBJETO correlaciones

@styler
def get_correlation(data: dict, activo: str):
    """
    Devuelve la matriz de correlaciones para un activo determinado.

    Args:
        data (dict): Diccionario con los datos de correlaciones de todos los activos.
        activo (str): Activo a buscar en el diccionario.

    Returns:
        DataFrame: Matriz de correlaciones para el activo.

    ACLARACION: El DataFrame es tipo Styler. Por lo que algunas funcionalidades propias de los DataFrame se pierden.
    """
    # Filter the data dictionary based on the activo
    filtered_data = {
        key: values[activo.upper()]
        for key, values in data.items()
    }
    # Create the DataFrame
    df = pd.DataFrame(filtered_data)
    df.columns.name = 'correlacion {}'.format(activo)
    return df

def crear_menu(id_search, activos, temporalidades):
    menu = ""

    for i in range(len(id_search)):
        assets = ' - '.join(activos[i])
        temps = ' '.join(temporalidades[i])
        menu_item = f'{id_search[i]}) {assets} : {temps}\n'
        menu += menu_item

    return menu


In [6]:
# PROGRAMA PRINCIPAL
# Load the data
date_path = os.path.join(os.getcwd(), 'data')

# Get the date directory
print(f' Directorios guardados: {os.listdir(date_path)}')

file_dir = os.path.join(date_path, os.listdir(date_path)[-1])#TODO: estamos eligiendo el ultimo directorio, HARDCODED es la carpeta del dia de hoy
#TODO: APLICAR crear_menu para elegir el directorio a cargar
print(f' Cantidad de solicitudes guardadas: {len(os.listdir(file_dir))//4}')

# obtener los nombres de los activos descargados
environment_temporalities = ['1d','4h','2h','1h','30m','15m','5m','3m', '1m']
elementos_SQ = [file for file in os.listdir(file_dir) if file.endswith('SQ.pickle')]

id_search, activos_encontrados, temporalidades = leer_nombre_ficheros(elementos_SQ, environment_temporalities)


if __name__ == '__main__':
    #TODO: crear una funcion main() que tenga argumentos de entrada, la fecha y luego los activos disponibles
    print(crear_menu(id_search, activos_encontrados, temporalidades))

    id_elegido = id_search[0]#TODO: acá va el input para cargar los datos correspondientes
    activos_elegidos = activos_encontrados[0]
    temporalidades_correspondientes = temporalidades[0]

    print(f' Activos elegidos: {activos_elegidos}')
    print(f' Temporalidades correspondientes: {temporalidades_correspondientes}')

    archivos_necesarios = ['.pickle','_SQ.pickle', '_CORR.pickle', '_TW.pickle']
    file_name_without_extension = f'{id_elegido}_' + '_'.join(activos_elegidos)+'_'+'_'.join(temporalidades_correspondientes)
    loaded_files = []
    for extension in archivos_necesarios:
        file_name = file_name_without_extension + extension
        file_path = os.path.join(file_dir, file_name)
        with open(file_path, 'rb') as file:
            loaded_files.append(pickle.load(file))
    data = {
        'data_raw': loaded_files[0],
        'data_SQ': loaded_files[1],
        'SQ_CORR': loaded_files[2],
        'time_window': loaded_files[3]
    }

 Directorios guardados: ['04_junio_2023', '05_junio_2023', '06_junio_2023', '07_junio_2023', '08_junio_2023']
 Cantidad de solicitudes guardadas: 2
1) BTC - XRP - APE : 4h 2h 1h 30m 15m 5m 3m 1m
2) ETH - LINK - BNB - NEO : 1d 4h 2h 1h

 Activos elegidos: ['BTC', 'XRP', 'APE']
 Temporalidades correspondientes: ['4h', '2h', '1h', '30m', '15m', '5m', '3m', '1m']


# 2) Mostrar las correlaciones

Podría utilizarse la libreria Rich para armar tablas y exportarlas en formato txt


In [12]:
if __name__ == '__main__':
    asset = 'btc'

    df = get_correlation(data['SQ_CORR'], asset)
    display(df)

correlacion btc,9,12,14,18,20,22,30,40,50,60,70,80,90
4h,0.70,0.60,0.54,0.59,0.62,0.63,0.57,0.34,0.31,0.52,0.37,0.14,0.04
2h,0.59,0.74,0.69,0.45,0.44,0.44,0.51,0.60,0.63,0.31,0.19,0.19,0.24
1h,0.63,0.68,0.59,0.59,0.65,0.69,0.75,0.43,0.32,0.54,0.58,0.62,0.63
30m,0.48,0.54,0.50,0.51,0.56,0.61,0.76,0.65,0.74,0.52,0.51,0.46,0.39
15m,0.66,0.60,0.44,0.41,0.41,0.42,0.41,0.58,0.68,0.43,0.50,0.57,0.62
5m,0.35,0.45,0.50,0.51,0.54,0.57,0.56,0.37,0.35,0.03,0.01,0.02,0.05
3m,0.32,0.36,0.27,0.39,0.43,0.46,0.61,0.58,0.56,0.40,0.37,0.35,0.36
1m,0.36,0.17,0.09,0.28,0.29,0.30,0.33,0.28,0.33,0.40,0.45,0.50,0.55


In [ ]:
# CREAR TABLA DE CORRELACIONES PARA EXPORTAR FORMATO TXT
# TODO: Obtener df en una variable tipo DataFrame, actualmente df es tipo Styler
## tabla para exportar a txt -> NO BORRAR!!
## tabla = tabulate(df, headers='keys', tablefmt='fancy_grid')
## print(f'Correlación {asset} con Squeeze Momentum Indicator')
## print(tabla)

In [15]:
# AUXILIAR PARA VER LA FLUCTUACION DE LAS MEDICIONES
# TODO: Manualmente leer dos archivos y ejecutarlos en celdas diferentes
if __name__ == '__main__':
    asset = 'btc'

    df = get_correlation(data['SQ_CORR'], asset)
    display(df)

Correlación XRP con Squeeze Momentum Indicator


correlacion XRP,9,12,14,18,20,22,30,40,50,60,70,80,90
4h,0.76,0.63,-0.11,0.16,0.18,0.22,0.32,0.52,0.57,0.56,0.58,0.60,0.63
2h,0.62,0.78,0.75,0.18,0.16,0.15,0.07,0.06,0.16,0.55,0.61,0.68,0.72
1h,0.65,0.64,0.41,0.61,0.67,0.72,0.77,0.33,0.15,0.13,0.19,0.27,0.36
30m,0.64,0.52,0.57,0.18,0.24,0.30,0.49,0.68,0.77,0.25,0.25,0.22,0.16
15m,0.33,0.50,0.63,0.56,0.56,0.56,0.58,0.25,0.39,0.46,0.56,0.66,0.73
5m,0.24,0.25,0.50,0.37,0.38,0.40,0.48,0.49,0.54,0.25,0.24,0.26,0.27
3m,0.52,0.53,0.43,0.39,0.44,0.49,0.62,0.43,0.51,0.46,0.52,0.54,0.53
1m,0.54,0.57,0.32,0.44,0.46,0.48,0.49,0.40,0.48,0.25,0.30,0.34,0.37


# 3) Crear un registro de las notas de las lecturas

In [ ]:
# Revisar tradegestor DEMO

# 1) La temporalidad mas correlacionada

# 2) Las temporalidades y los len que se van a analizar

# 3) La temporalidad y los len que se van a visualizar

# 4) Si se va a tomar una decisión dentro del mercado al respecto

# 5) Comentarios que se quieran hacer respecto al DI y al ADX

# 4) Visualización de la data

In [55]:
# Una forma auxiliar de imprimir dataframes
# La previsualización de los datos asociados a los axis tienen que ser visualizados de esta manera

import plotly.graph_objects as go
from rich import print
from datetime import datetime

#print(f'Correlación {asset} con Squeeze Momentum Indicator')
#print(df.loc[:,70:])
#print(df.loc[:,30:60])
#print(df.loc[:,9:22])


In [93]:
def candlestick_squeeze_plot(data, asset, temp, period, period_window):
    features = ['open', 'close', 'high', 'low', 'value']
    df = data.loc[period_window:,features]

    dist_tick = len(df)//7

    candlestick = go.Candlestick(x=df.index,
                    open=df['open'],
                    high=df['high'],
                    low=df['low'],
                    close=df['close'])

    bar = go.Bar(x=df.index, y=df['value'], name='squeeze')

    fig = go.Figure(data=[candlestick])
    fig2 = go.Figure(data=[bar])
    fig.layout.xaxis.type = 'category'
    fig.layout.xaxis.rangeslider.visible = False
    fig.update_layout(
        xaxis = dict(
            tickmode = 'array',
            tickvals = df.index[dist_tick//2::dist_tick],#TODO:considerar este slide para balancear los ticks del eje x
            ticktext = df.index[:-dist_tick//2:dist_tick],#TODO:considerar este slide para balancear los ticks del eje x
        ),
        autosize=False,
        width=1200,
        height=600,
        margin=dict(
            
            l=50,
            r=50,
            b=50,
            t=50,
            pad=4
        ),
        paper_bgcolor="LightSteelBlue",
        title=f'{asset} {temp} | len squeeze: {period}'
    )

    fig2.update_layout(
        autosize=False,
        width=1200,
        height=300,
        margin=dict(
            
            l=50,
            r=50,
            b=1,
            t=10,
            pad=4
        ),
        paper_bgcolor="LightSteelBlue",
    )

    fig.show()
    fig2.show() 


In [96]:
period = 14
asset = 'BTC'
temp = '1h'

period_window = data['time_window'][temp][period]
df = data['data_SQ'][period][asset][temp]

candlestick_squeeze_plot(df, asset, temp, period, period_window)


period = 50
asset = 'BTC'
temp = '1h'

period_window = data['time_window'][temp][period]
df = data['data_SQ'][period][asset][temp]

candlestick_squeeze_plot(df, asset, temp, period, period_window)

'2023-06-05 16:44:20'

# 5) Exportar tablas, registros y visualizaciones